# Downloads and Imports

In [1]:
!pip install -q transformers
!pip install -q sentencepiece


     |████████████████████████████████| 5.3 MB 5.2 MB/s 
     |████████████████████████████████| 163 kB 63.6 MB/s 
     |████████████████████████████████| 7.6 MB 46.6 MB/s 
     |████████████████████████████████| 1.3 MB 4.9 MB/s 


In [2]:
#Data
from google.colab import drive
import sqlite3
import json
#NEE
import spacy
from spacy.lang.en import English
from spacy.pipeline import EntityRuler
import re
from spacy import displacy
from collections import Counter
import en_core_web_sm
#Models
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch


# Data

In [3]:
drive.mount("drive")

Mounted at drive


In [4]:
!cp ./drive/MyDrive/FDB.zip  ./ 

In [5]:
!unzip  FDB.zip

Archive:  FDB.zip
  inflating: FDB.db                  
  inflating: FDB.sqbpro              


In [6]:
con = sqlite3.connect("FDB.db")


In [7]:
sql_query = """SELECT name FROM sqlite_master  
  WHERE type='table';"""
cursor =  con.cursor()

cursor.execute(sql_query)
tables = [ x[0]  for x in cursor.fetchall()]

In [8]:
tables

['us_equities_news_dataset', 'nasdaq_screener', 'stockHistory']

In [9]:
sql_query = """SELECT * FROM 'us_equities_news_dataset'  """
cursor.execute(sql_query)
news = cursor.fetchall()

import pandas as pd
# Create your connection.
dfs = {}
for t in tables:
  dfs[t] = pd.read_sql_query(f"SELECT * FROM {t}", con)

# Summarization Model

Haven't yet finished but used it to compare applying NEE in both the whole and summatized text



In [79]:
model_name = "google/pegasus-cnn_dailymail"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(device)

Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [80]:
src_text = [
 '''The highlight of the past week was the start of deliveries of Model 3 vehicles  built at the Gigafactory3 in Shanghai  by Tesla 
  NASDAQ TSLA    The first 15 customers  who are slated to get their Model 3 cars  are employees of Tesla  Mass deliveries of China made Model
   3 are expected to begin from Jan 7 Tesla s Chinese rival NIO Inc    NYSE NIO   reported narrower than expected loss in third quarter 2019 on 
   the back of rising deliveries of the ES6 model  Nissan Motor   OTC NSANY   also made headlines as itsvice chief operating officer  
   Jun Seki abruptly resigned from the company  Seki s decision to resign is seen as yet another potential blow to Nissan s turnaround plan 
    Read the Last Auto Stock Roundup   Recap of the Week s Most Important Stories1  Reportedly  Tesla will commence mass deliveries of Model 3 
    vehicles built at its Gigafactory3 in Shanghai from Jan 7  2020 Notably  Gigafactory3 is China s first completely foreign owned car plant  
    Tesla started construction of the plant in January and began production in October  The Gigafactory3 in Shanghai has been built as part of Tesla 
    s plan to boost presence in the world s largest auto market  China  and reduce the impact of the U S  China trade war  The Ministry of Industry 
    and Information Technology recently stated that it added China made Model 3 of Tesla to a list of new energy vehicles that are excluded from 
    purchase tax  Tesla currently carries a Zacks Rank  2  Buy   You can see  2  NIO reported adjusted loss per share of 33 cents in third quarter 
    2019  narrower than the Zacks Consensus Estimate of a loss of 41 cents and the prior year quarter s loss of  1 51  High demand for the company s
     ES6 model led to the improved results  Notably  total deliveries in the quarter under review were 4 799 units  suggesting an uptick from 3 268 units in the year ago period  Total revenues grew 25  year over year to  257 million  The figure also outpaced the Zacks Consensus Estimate of  249 million  NIO anticipates vehicle deliveries to be more than 8 000 units for fourth quarter 2019  suggesting a rise of more than 66 7  from the prior year period   Read more   3  Nissan Motor s third highest ranked executive and vice chief operating officer  Jun Seki  abruptly resigned from the company just weeks after taking up the job  Resultantly  the company s shares fell to an eight year low  declining 3 1  to 633 yen as of Dec 25  2019  in the Tokyo Stock Exchange  The company is also struggling to recover from the ousting of former chairman Carlos Ghosn  industry downturn  decade low profits and strained ties with French partner Renault SA  PA RENA   Following the surprise resignation of Seki  Nissan has chosen executive vice president Hideyuki Sakamoto as a board candidate   Read more   4  Volkswagen  DE VOWG p  AG   OTC VWAGY   announced that it will reach the target of 1 million electric cars by the end of 2023  which is two years earlier than anticipated  As a result  the company significantly raised the electric car production target for 2025  It now anticipates producing 1 5 million electric cars by 2025 In November  the company started production of the ID 3 model at the Zwickau plant  It expects to produce up to 330 000 EVs from 2021  making Zwickau the largest and most efficient EV factory in Europe  Volkswagen anticipates that the first ID 3 electric cars will appear on Europe s roads in the summer of 2020  The company plans to invest  33 billion in e mobility throughout the group by 2024  which includes  11 billion in the Volkswagen brand   Read more  TGT  for Electric Cars 5  Advance Auto Parts  Inc    NYSE AAP   recently announced that it has purchased the DieHard car battery brand from Transform Holdco LLC  the parent company of Kmart and Sears  for a cash consideration of  200 million Following the acquisition  Advance Auto Parts will have the right to sell DieHard batteries  and extend the DieHard brand to other automotive and vehicle categories  Moreover  the acquisition of DieHard is expected to differentiate Advance Auto Parts  drive increased DIY customer traffic  create a unique value proposition for expert customers and independent Carquest partners  as well as drive significant long term shareholder value   Read more   Price Performance  The following table shows the price movement of some of the major auto players over the past week and six month period In the past week  Harley Davidson   NYSE HOG   has declined the most  while General Motors  NYSE GM  has been the maximum gainer  Over the past six months  Tesla has been the best performer  having rallied 83 1   while Ford   NYSE F   has been the worst performing stock What s Next in the Auto Space Domestic and foreign auto giants including General Motors  Ford  Fiat Chrysler and Toyota Motors  among others  are set to soon report full year 2019 U S  car sales Biggest Tech Breakthrough in a Generation Be among the early investors in the new type of device that experts say could impact society as much as the discovery of electricity  Current technology will soon be outdated and replaced by these new devices  In the process  it s expected to create 22 million jobs and generate  12 3 trillion in activity A select few stocks could skyrocket the most as rollout accelerates for this new tech  Early investors could see gains similar to buying Microsoft  NASDAQ MSFT  in the 1990s  Zacks  just released special report reveals 8 stocks to watch  The report is only available for a limited time''']
batch = tokenizer(src_text, truncation=True, padding='longest', return_tensors="pt").to(device)
translated = model.generate(**batch)
tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
print(tgt_text)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1301: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 128 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


['The highlight of the past week was the start of deliveries of Model 3 vehicles built at the Gigafactory3 in Shanghai by Tesla NASDAQ TSLA.<n>Chinese rival NIO Inc NYSE NIO reported narrower than expected loss in third quarter 2019 on the back of rising deliveries of the ES6 model Nissan Motor OTC NSANY.<n>Volkswagen DE VOWG p AG OTC VWAGY announced that it will reach the target of 1 million electric cars by the end of 2023 which is two years earlier than anticipated.']


# Named Entity Extraction
  extracting the mentioned companies from the articles to construct edges for the KG

## helper functions

In [75]:
def load_data(file):
    with open(file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

def save_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


def generate_better_names(file):
    data = load_data(file)
    updated_names = []
    #Remove unneeded words from the company name
    words = ["Company", "Corporation ", "Inc.", "Inc", "Technology", "Common Stock", "Group"]
    for item in data:
      itemList =[]
      itemList.append(item)

      #Some companies are called and mentioned with only its first name 
      item1 = item
      item1 = item1.replace("The", "")
      if(item1[-1]==" "):
        item1= item1[:-1] 
      itemList.append(item1.split(" ")[0])

      for word in words:
        item = item.split(word, 1)[0]
      item = re.sub("\(.*?\)|\[.*?\]","", item)
      if(item[-1]==" "):
        item= item[:-1]
      itemList.append(item)
      itemList = list(set(itemList))
      updated_names.append(itemList)
    
    return updated_names

def create_training_data(file, typeo):
    opened_file = open(file)
    data = json.load(opened_file)
    patterns = []
    for index, itemList in enumerate(data):
      for item in itemList:
          pattern = {
                      "label": typeo[index],
                      "pattern": item
                      }
          patterns.append(pattern)
    for item in typeo:
      pattern = {
                  "label": item,
                  "pattern": item
                  }

      patterns.append(pattern)
    return (patterns)

def generate_rules(patterns):
    nlp=English()
    ruler = nlp.add_pipe("entity_ruler")
    ruler.add_patterns(patterns)
    nlp.to_disk("trained_ner")

def test_model(text):
    nlp =spacy.load("trained_ner")
    doc = nlp(text)
    results = []
    for ent in doc.ents:
        results.append(ent.label_)
    return list((set(results)))


## Preprocessing on companies' names

In [61]:
save_data("companies_names.json",json.dumps(list(dfs['nasdaq_screener']['Name'].unique()), indent=2) )

In [56]:
updated_names = generate_better_names("companies_names.json")
save_data("updated_names.json",updated_names)


[['NIO', 'NIO Inc. American depositary shares each  representing one Class A ordinary share'], ['Uber', 'Uber Technologies', 'Uber Technologies Inc. Common Stock'], ['Intel Corporation Common Stock', 'Intel'], ['Microsoft', 'Microsoft Corporation Common Stock'], ['Micron Technology Inc. Common Stock', 'Micron'], ['Ballard Power Systems Inc. Common Shares', 'Ballard Power Systems', 'Ballard'], ['Freeport-McMoRan', 'Freeport-McMoRan Inc. Common Stock'], ['EQT Corporation Common Stock', 'EQT'], ['Marvell', 'Marvell Technology Inc. Common Stock'], ['Truist Financial', 'Truist', 'Truist Financial Corporation Common Stock'], ['Masco', 'Masco Corporation Common Stock'], ['KLA Corporation Common Stock', 'KLA'], ['Eversource Energy ', 'Eversource Energy (D/B/A) Common Stock', 'Eversource'], ['L3Harris Technologies Inc. Common Stock', 'L3Harris Technologies', 'L3Harris'], ['Nucor Corporation Common Stock', 'Nucor'], ['Chubb', 'Chubb Limited  Common Stock', 'Chubb Limited '], ['Alnylam', 'Alnylam

In [62]:
patterns=create_training_data("updated_names.json", list(dfs['nasdaq_screener']['Symbol']))
generate_rules(patterns)

In [65]:
save_data("patterns.json", patterns)

In [66]:
import os.path
from os import path

pathv = "/content/drive/MyDrive/Data"
if path.exists('/content/drive/MyDrive/GP_Data') == False:
  os.mkdir(pathv)

!cp -r "/content/companies_names.json" "/content/drive/MyDrive/GP_Data"
!cp -r "/content/updated_names.json" "/content/drive/MyDrive/GP_Data"
!cp -r "/content/patterns.json" "/content/drive/MyDrive/GP_Data"
!cp -r "/content/trained_ner" "/content/drive/MyDrive/GP_Data"

## Cutomized NER Model

In [76]:
test_model('''FedEx  NYSE FDX  the transport giant reports their fiscal Q2  20 financial results after the bell on Tuesday  December 17  with consensus Wall Street estimates being  2 76 in earnings per share  EPS  on  17 58 billion in revenue for expected year over year declines of 32  and 1  respectively 
The good news is that  starting with the November  19 earnings release  FedEx  NYSE FDX  begins to lap the awful compares of the last 4 quarters 
The peak in FDX s stock occurred in late January  early February  18 when the stock peaked at  274 per share but since then the China trade and tariffs issue  the supply chain rerouting that has gripped industrial America  the slowdown in Europe on trade and Brexit issues  and the growing competition to FedEx  NYSE FDX  Ground from the Amazon  NASDAQ AMZN  Prime trucks and one day delivery has taken a significant toll on the stock  That doesn t even include the hack of FedEx s biggest European acquisition  TNT  which impacted that business and it s integration into FedEx as a whole   The TNT integration was scheduled to take 12 quarters or 3 years  so it will be interesting to hear where Fred Smith  CEO  and Alan Graf  CFO  think that stands today  According to sell side comments a few quarters ago  the integration should be in the 9th or 10th quarter  

As readers can see  the fiscal 2020  and 2021 EPS estimates peaked in early 2018  a few months after the stock peaked and have fallen dramatically since  Have the EPS estimates bottomed   There is usually a lag between the stock price peak or trough and the trend in estimates so be patient  but with the trade deal and Brexit scheduled for January 31  2020 investors and CEO s have more clarity on trade and Brexit rules and thus there should be some pickup in international trade 

To give readers and investors an idea of the operating leverage in FDX  revenue estimates haven t declined nearly as much as EPS estimates  which is typical with businesses with high fixed costs 
The decline in revenue estimates from early 2018 for fiscal 2020  has been less than 3  versus the 40  decline in the fiscal 2020 EPS estimate 
The Issues
The obvious issues like the China tariff tiff and the never ending Brexit debate are now either behind us in the form of Phase One of the China Trade deal AND the USMCA agreed to last week  or will soon be settled as Boris Johnson has set a firm date of January 31  2020 for Brexit to happen 
While the trade deals impact FedEx  NYSE FDX  Express   52  of revenue but operating income has fallen to just 29  of FedEx s total operating income as of Q1  20  FedEx Ground and the encroachment by Amazon  NASDAQ AMZN  is a little worrisome  Fred Smith has described the worry as  fantastical  years ago  but Amazon with 1 day delivery seems to have upped the ante  and pressure on Ground  which was 30  of total FedEx revenue as of Q1  20 but 66  of operating income   FedEx took a charge for TNT integration  which I assume would have been flushed through Express  hence the drop in Express operating income in Q1  20  
The other issue for FedEx is free cash flow generation  FedEx s cash flow and free cash flow is tracked every quarter on the valuation spreadsheet  acquisitions are INCLUDED in capex for free cash flow calculation purposes  and 9 of the last 12 quarters for FedEx has been free cash flow negative on a trailing 4 quarter basis  Q1  20  which was the quarter ended Aug  19 saw a free cash flow deficit of  850 million  which is sizable  although to be evenhanded with FedEx  the first fiscal quarter of every year is typically free cash negative 

FedEx has moved back over its 200 day moving average which is a key level for chart watchers  Let s see if  post earnings  the stock can stay above the 200 day and 50 week moving average  FedEx traded as low the high  140 s after December  2018 s earnings release  so a trade below  145 wouldn t want to be seen by those who are long the stock 
Conclusion
After falling almost 50  from its January  18 highs  FedEx  NYSE FDX  is trying to steady the ship and the trade deals   China and USMCA  and ultimately Brexit should provide substantial clarity to an international trade environment that was murky at best and completely unnavigable at worst   How as a CEO do you plan for the next 1 year or 3 year capex when you have no idea what the trade rules or reg s will look like   
The newer problem for FedEx  NYSE FDX  is now what about the domestic US Ground business and will that be the albatross that slows the Ground giant  
I was out for dinner last week at a restaurant in the western suburbs of Chicago talking to an old trucker guy  older than me  owned a small trucking company his whole life  and he made the point that Amazon  NASDAQ AMZN  would not have gotten into the business without the goal of dominating it eventually 
However reading the August  19  quarterly conference call notes  Fred Smith gave the metrics on the Ground and package business and it sounded like FedEx  NYSE FDX  was viewing Ground as a continuing opportunity rather than a threat  Hearing about businesses like Uber  Freight and other start ups as I see more and more Amazon  NASDAQ AMZN  Prime trucks rolling around Chicago and the  burbs  has me nervous for sure 
Clients have less than a 1  position in FedEx  NYSE FDX  that s been built over the last year  which includes a  36 cost basis position from March of 2000  More FedEx will be added here   maybe even a little before the Tuesday s results   since i think Fred Smith and Alan Graf are one of the best management teams in the Transportation sector 
FedEX is trading at less than 1x revenue  8x cash flow and 14x 2020 s current estimate of  12 
What will be watched after the report  and assume it will be bad news  but the comparisons get far easier in fiscal 2020 and fiscal 2021  will be the trend and magnitude of EPS and revenue revisions  what Fred Smith and Alan Graf might say about the signing of the trade deals and how fast some of this traffic can recover  and what free cash flow guidance might look like 
Readers should assume the numbers will continue to be bad for the next two quarters 
Thanks for reading''')

['JNJ', 'UBER', 'AMX', 'AMZN', 'CGA', 'FDX']

In [77]:
#Mentioned companies from the whole text
test_model('''The highlight of the past week was the start of deliveries of Model 3 vehicles  built at the Gigafactory3 in Shanghai  by Tesla 
  NASDAQ TSLA    The first 15 customers  who are slated to get their Model 3 cars  are employees of Tesla  Mass deliveries of China made Model
   3 are expected to begin from Jan 7 Tesla s Chinese rival NIO Inc    NYSE NIO   reported narrower than expected loss in third quarter 2019 on 
   the back of rising deliveries of the ES6 model  Nissan Motor   OTC NSANY   also made headlines as itsvice chief operating officer  
   Jun Seki abruptly resigned from the company  Seki s decision to resign is seen as yet another potential blow to Nissan s turnaround plan 
    Read the Last Auto Stock Roundup   Recap of the Week s Most Important Stories1  Reportedly  Tesla will commence mass deliveries of Model 3 
    vehicles built at its Gigafactory3 in Shanghai from Jan 7  2020 Notably  Gigafactory3 is China s first completely foreign owned car plant  
    Tesla started construction of the plant in January and began production in October  The Gigafactory3 in Shanghai has been built as part of Tesla 
    s plan to boost presence in the world s largest auto market  China  and reduce the impact of the U S  China trade war  The Ministry of Industry 
    and Information Technology recently stated that it added China made Model 3 of Tesla to a list of new energy vehicles that are excluded from 
    purchase tax  Tesla currently carries a Zacks Rank  2  Buy   You can see  2  NIO reported adjusted loss per share of 33 cents in third quarter 
    2019  narrower than the Zacks Consensus Estimate of a loss of 41 cents and the prior year quarter s loss of  1 51  High demand for the company s
     ES6 model led to the improved results  Notably  total deliveries in the quarter under review were 4 799 units  suggesting an uptick from 3 268 units in the year ago period  Total revenues grew 25  year over year to  257 million  The figure also outpaced the Zacks Consensus Estimate of  249 million  NIO anticipates vehicle deliveries to be more than 8 000 units for fourth quarter 2019  suggesting a rise of more than 66 7  from the prior year period   Read more   3  Nissan Motor s third highest ranked executive and vice chief operating officer  Jun Seki  abruptly resigned from the company just weeks after taking up the job  Resultantly  the company s shares fell to an eight year low  declining 3 1  to 633 yen as of Dec 25  2019  in the Tokyo Stock Exchange  The company is also struggling to recover from the ousting of former chairman Carlos Ghosn  industry downturn  decade low profits and strained ties with French partner Renault SA  PA RENA   Following the surprise resignation of Seki  Nissan has chosen executive vice president Hideyuki Sakamoto as a board candidate   Read more   4  Volkswagen  DE VOWG p  AG   OTC VWAGY   announced that it will reach the target of 1 million electric cars by the end of 2023  which is two years earlier than anticipated  As a result  the company significantly raised the electric car production target for 2025  It now anticipates producing 1 5 million electric cars by 2025 In November  the company started production of the ID 3 model at the Zwickau plant  It expects to produce up to 330 000 EVs from 2021  making Zwickau the largest and most efficient EV factory in Europe  Volkswagen anticipates that the first ID 3 electric cars will appear on Europe s roads in the summer of 2020  The company plans to invest  33 billion in e mobility throughout the group by 2024  which includes  11 billion in the Volkswagen brand   Read more  TGT  for Electric Cars 5  Advance Auto Parts  Inc    NYSE AAP   recently announced that it has purchased the DieHard car battery brand from Transform Holdco LLC  the parent company of Kmart and Sears  for a cash consideration of  200 million Following the acquisition  Advance Auto Parts will have the right to sell DieHard batteries  and extend the DieHard brand to other automotive and vehicle categories  Moreover  the acquisition of DieHard is expected to differentiate Advance Auto Parts  drive increased DIY customer traffic  create a unique value proposition for expert customers and independent Carquest partners  as well as drive significant long term shareholder value   Read more   Price Performance  The following table shows the price movement of some of the major auto players over the past week and six month period In the past week  Harley Davidson   NYSE HOG   has declined the most  while General Motors  NYSE GM  has been the maximum gainer  Over the past six months  Tesla has been the best performer  having rallied 83 1   while Ford   NYSE F   has been the worst performing stock What s Next in the Auto Space Domestic and foreign auto giants including General Motors  Ford  Fiat Chrysler and Toyota Motors  among others  are set to soon report full year 2019 U S  car sales Biggest Tech Breakthrough in a Generation Be among the early investors in the new type of device that experts say could impact society as much as the discovery of electricity  Current technology will soon be outdated and replaced by these new devices  In the process  it s expected to create 22 million jobs and generate  12 3 trillion in activity A select few stocks could skyrocket the most as rollout accelerates for this new tech  Early investors could see gains similar to buying Microsoft  NASDAQ MSFT  in the 1990s  Zacks  just released special report reveals 8 stocks to watch  The report is only available for a limited time''')

['A', 'DE', 'TSLA', 'TGT', 'CAAS', 'GM', 'TM', 'HOG', 'NIO', 'CGA', 'MSFT']

In [81]:
test_model(tgt_text[0])  #Mentioned companies from the summurized text 

['DE', 'NIO', 'TSLA']